# Segmenting and Clustering Neighborhoods in Toronto
## Coursera's "Applied Data Science Capstone" Project - Week 3 Assignment

**Student:** Michael Onishi <br>
**Date:** June 2020

https://www.coursera.org/learn/applied-data-science-capstone

In [1]:
import pandas as pd
import numpy as np

> Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

<img src="https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/7JXaz3NNEeiMwApe4i-fLg_40e690ae0e927abda2d4bde7d94ed133_Screen-Shot-2018-06-18-at-7.17.57-PM.png?expiry=1592611200000&hmac=5arD4dNVCUXq6z4BZyVFNxp6G9yPUeYtOloI7T1HqfA" width="500">

In [2]:
# only getting the first table in the html
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", header=0)[0]

In [3]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
df.shape

(180, 3)

> Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [5]:
df = df[df['Borough'] != 'Not assigned']

> More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [6]:
# checking if there are rows with same postal code
df.groupby('Postal Code').filter(lambda x: len(x) > 1)

,Postal Code,Borough,Neighborhood


There is no rows with same postal code, but the code below would do the job

In [7]:
df = df.groupby(['Postal Code','Borough'])['Neighborhood'].apply(', '.join).reset_index()

> If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [8]:
# checking if there is any row with "Not assigned" Neighborhood

df[df['Neighborhood'] == 'Not assigned']

,Postal Code,Borough,Neighborhood


No Neighborhood with "Not assigned" value.

In [9]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
df.shape

(103, 3)

> Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

For this task, I will get latitude and longitude information using the [pgeocode library](https://pypi.org/project/pgeocode/) , that gets geocodes from postal code.

In [11]:
import pgeocode
geolocator = pgeocode.Nominatim('ca')

In [12]:
lats, longs = [], []
for _, row in df.iterrows():
    location = geolocator.query_postal_code(row['Postal Code'])
    lats.append(location.latitude)
    longs.append(location.longitude)

In [13]:
df['Latitude'] = lats
df['Longitude'] = longs

In [14]:
df.shape

(103, 5)

Checking if there is any latitude or longitude unfilled.

In [15]:
df[df['Latitude'].isna() | df['Longitude'].isna()]

,Postal Code,Borough,Neighborhood,Latitude,Longitude
86,M7R,Mississauga,Canada Post Gateway Processing Centre,NaN,NaN


There is only one row without latitude or longitude information. So I will drop it.

In [16]:
df.dropna(subset = ['Latitude', 'Longitude'], inplace=True)

In [17]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.8113,-79.1930
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.7878,-79.1564
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7678,-79.1866
3,M1G,Scarborough,Woburn,43.7712,-79.2144
4,M1H,Scarborough,Cedarbrae,43.7686,-79.2389


In [18]:
df.shape

(102, 5)

> Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.
> 
> Just make sure:
> 
> 1. to add enough Markdown cells to explain what you decided to do and to report any observations you make.
> 1. to generate maps to visualize your neighborhoods and how they cluster together.

#### Creating a map of Toronto with neighborhoods superimposed on top.

In [19]:
import folium

In [20]:
# create map of Toronto using latitude and longitude values
toronto_latitude = 43.651070
toronto_longitude = -79.347015
map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        location=(lat, lng),
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

As suggested, I will work only with boroughs that contain the word Toronto.

In [21]:
toronto_data = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
print(toronto_data.shape)
toronto_data.head()

(39, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.6784,-79.2941
1,M4K,East Toronto,"The Danforth West, Riverdale",43.6803,-79.3538
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.6693,-79.3155
3,M4M,East Toronto,Studio District,43.6561,-79.3406
4,M4N,Central Toronto,Lawrence Park,43.7301,-79.3935


Let's visualize a map only with these boroughs

In [22]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [23]:
import yaml
import requests

## retrieving the Foursquare API credentials
with open("config.yml", 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)

In [24]:
CLIENT_ID = cfg['foursquare_api_credentials']['CLIENT_ID'] # your Foursquare ID
CLIENT_SECRET = cfg['foursquare_api_credentials']['CLIENT_SECRET'] # your Foursquare Secret
VERSION = '20200618' # Foursquare API version

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

Let's check the size of the resulting dataframe

In [27]:
print(toronto_venues.shape)
toronto_venues.head()

(1537, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.6784,-79.2941,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.6784,-79.2941,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.6784,-79.2941,Beaches Bake Shop,43.680363,-79.289692,Bakery
3,The Beaches,43.6784,-79.2941,The Beech Tree,43.680493,-79.288846,Gastropub
4,The Beaches,43.6784,-79.2941,Grover Pub and Grub,43.679181,-79.297215,Pub


Let's check how many venues were returned for each neighborhood

In [28]:
toronto_venues.groupby('Neighborhood').size().to_frame(name='Count')

,Count
Neighborhood,
Berczy Park,92
"Brockton, Parkdale Village, Exhibition Place",39
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",13
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",58
Central Bay Street,62
Christie,12
Church and Wellesley,80
"Commerce Court, Victoria Hotel",100
Davisville,23


Let's find out how many unique categories can be curated from all the returned venues

In [29]:
print(f'There are {len(toronto_venues["Venue Category"].unique())} uniques categories.')

There are 218 uniques categories.


## Analyzing Each Neighborhood

In [30]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
toronto_onehot.insert(0, 'Neighborhood', toronto_venues['Neighborhood'])

toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [31]:
toronto_onehot.shape

(1537, 218)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [32]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.010870,0.021739,0.000000,0.000000,0.000000,0.000000,0.010870,...,0.000000,0.010870,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.010870
1,"Brockton, Parkdale Village, Exhibition Place",0.025641,0.0000,0.000000,0.025641,0.000000,0.025641,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.076923
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,...,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.00,0.017241
4,Central Bay Street,0.000000,0.0000,0.000000,0.000000,0.016129,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.016129,0.016129,0.000000,0.016129,0.000000,0.000000,0.00,0.000000
5,Christie,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Church and Wellesley,0.000000,0.0125,0.012500,0.000000,0.000000,0.012500,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012500,0.000000,0.00,0.025000
7,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.030000,0.010000,0.000000,0.000000,0.030000,0.000000,0.000000,...,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.01,0.000000
8,Davisville,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Davisville North,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000


Let's confirm the new size

In [33]:
toronto_grouped.shape

(39, 218)

Let's print each neighborhood along with the top 5 most common venues

In [34]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print(f"----{hood}----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.10
1               Hotel  0.05
2                Café  0.05
3              Bakery  0.03
4  Seafood Restaurant  0.03


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0                    Café  0.08
1             Coffee Shop  0.08
2          Breakfast Spot  0.05
3  Thrift / Vintage Store  0.05
4               Gift Shop  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0          Restaurant  0.15
1         Coffee Shop  0.15
2         Yoga Studio  0.08
3  Italian Restaurant  0.08
4    Sushi Restaurant  0.08


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0         Coffee Shop  0.07
1  Italian Restaurant  0.05
2                Café  0.05
3                 Bar  0.05
4   French Restaurant  0.03


----Central Bay

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append(f'{ind+1}{indicators[ind]} Most Common Venue')
    except:
        columns.append(f'{ind+1}th Most Common Venue')

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Hotel,Restaurant,Cocktail Bar,Beer Bar,Bakery,Japanese Restaurant,Seafood Restaurant,Pub
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Thrift / Vintage Store,Breakfast Spot,Gift Shop,Dance Studio,Brewery,Sandwich Place,Chiropractor,Restaurant
2,"Business reply mail Processing Centre, South C...",Restaurant,Coffee Shop,Yoga Studio,Sushi Restaurant,Italian Restaurant,Bank,Bookstore,Japanese Restaurant,Breakfast Spot,Deli / Bodega
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Italian Restaurant,Café,Bar,Park,Speakeasy,Restaurant,Bakery,Bank,Gym / Fitness Center
4,Central Bay Street,Coffee Shop,Sandwich Place,Japanese Restaurant,Bubble Tea Shop,Italian Restaurant,Café,Middle Eastern Restaurant,Park,Shopping Mall,Shoe Store


## Clustering Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [37]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=42).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [38]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_data.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.6784,-79.2941,0,Pub,Health Food Store,Bakery,Trail,Gastropub,Curling Ice,Dance Studio,Farmers Market,Falafel Restaurant,Event Space
1,M4K,East Toronto,"The Danforth West, Riverdale",43.6803,-79.3538,0,Greek Restaurant,Ice Cream Shop,Café,Italian Restaurant,Cosmetics Shop,Bubble Tea Shop,Spa,Restaurant,Cocktail Bar,Coffee Shop
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.6693,-79.3155,0,Fast Food Restaurant,Restaurant,Movie Theater,Italian Restaurant,Steakhouse,Coffee Shop,Ice Cream Shop,Pub,Fish & Chips Shop,Brewery
3,M4M,East Toronto,Studio District,43.6561,-79.3406,0,Coworking Space,Park,Diner,Baseball Field,Coffee Shop,Garden Center,Performing Arts Venue,Gym,Distribution Center,Falafel Restaurant
4,M4N,Central Toronto,Lawrence Park,43.7301,-79.3935,2,Photography Studio,Park,Distribution Center,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


Finally, let's visualize the resulting clusters

In [39]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

#### Cluster 1

This is by far the dominating cluster and they are located more in the central areas. This is caracterized by having many coffee shops, bar and restaurants.

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,0,Pub,Health Food Store,Bakery,Trail,Gastropub,Curling Ice,Dance Studio,Farmers Market,Falafel Restaurant,Event Space
1,"The Danforth West, Riverdale",0,Greek Restaurant,Ice Cream Shop,Café,Italian Restaurant,Cosmetics Shop,Bubble Tea Shop,Spa,Restaurant,Cocktail Bar,Coffee Shop
2,"India Bazaar, The Beaches West",0,Fast Food Restaurant,Restaurant,Movie Theater,Italian Restaurant,Steakhouse,Coffee Shop,Ice Cream Shop,Pub,Fish & Chips Shop,Brewery
3,Studio District,0,Coworking Space,Park,Diner,Baseball Field,Coffee Shop,Garden Center,Performing Arts Venue,Gym,Distribution Center,Falafel Restaurant
5,Davisville North,0,Food & Drink Shop,Park,Breakfast Spot,Bus Line,Department Store,Dog Run,Gym / Fitness Center,Dance Studio,Deli / Bodega,Cupcake Shop
7,Davisville,0,Café,Sandwich Place,Italian Restaurant,Dessert Shop,Coffee Shop,Gym,Thai Restaurant,Farmers Market,Fast Food Restaurant,Seafood Restaurant
9,"Summerhill West, Rathnelly, South Hill, Forest...",0,Light Rail Station,Coffee Shop,Liquor Store,Supermarket,Yoga Studio,Doner Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
11,"St. James Town, Cabbagetown",0,Coffee Shop,Pizza Place,Bakery,Café,Italian Restaurant,Park,Restaurant,Convenience Store,Chinese Restaurant,Butcher
12,Church and Wellesley,0,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Restaurant,Gay Bar,Yoga Studio,Grocery Store,Men's Store,Café,Hotel
13,"Regent Park, Harbourfront",0,Coffee Shop,Breakfast Spot,Yoga Studio,Thai Restaurant,Bakery,Beer Store,Distribution Center,Electronics Store,Event Space,Food Truck


#### Cluster 2

This is a cluster of only the Roselawn Neighborhood. Probably because of the very specific combination of venues: Home Service, Yoga Studio, Dog Run. This Neighborhood is not in the central area.

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Roselawn,1,Home Service,Yoga Studio,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


#### Cluster 3

Another cluster of only the one Neighborhood: Lawrence Park. Again, probably because of the very specific combination of venues: Photography Studio, Distribution Center. This Neighborhood is not in the central area.

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Lawrence Park,2,Photography Studio,Park,Distribution Center,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


#### Cluster 4

This cluster is in the vinicity of the central area and grouped together Neighborhoods with Parks, Playgrounds, Gym.

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,"North Toronto West, Lawrence Park",3,Gym Pool,Garden,Playground,Park,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
8,"Moore Park, Summerhill East",3,Park,Thai Restaurant,Grocery Store,Tennis Court,Gym,Falafel Restaurant,Farmers Market,Event Space,Discount Store,Electronics Store
10,Rosedale,3,Playground,Grocery Store,Park,Candy Store,Yoga Studio,Dog Run,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
34,"High Park, The Junction South",3,Park,Bowling Alley,Residential Building (Apartment / Condo),Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


#### Cluster 5

Another cluster of only the one Neighborhood: Forest Hill North & West, Forest Hill Road Park. Probably because of Trails and Parks.

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,"Forest Hill North & West, Forest Hill Road Park",4,Park,Trail,Yoga Studio,Distribution Center,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


## Conclusion

The clustering in this case was not so informative, because 3 out of 5 clusters had only one Neighborhood. This may be because there are too many features (categories of venues) and the Neighborhoods in general shared many similarities in only a few of them, so the Neighborhoods that had just a little different profile got separated.

Another hypothesis is that in the central and downtown areas, the Neighborhoods are very close together, making it very similar and grouped together naturally.